# Python의 기본 개념 익히기

In [ ]:
A = 1

In [ ]:
A += 1

In [ ]:
print(A)

In [ ]:
B = 2

In [ ]:
B += A

In [ ]:
print(B)

In [ ]:
목록 = ['경복궁', '광화문', '시청']

In [ ]:
print(목록[0])

In [ ]:
for 인자 in 목록:
    print(인자)

# Pandas : 실거래가(부동산 가격) 변화 분석

In [ ]:
# import pandas as pd에서 에러가 뜰 경우
# !pip install pandas
import pandas as pd

In [ ]:
아파트 = pd.read_csv('아파트(매매)__실거래가_201908.csv', engine='python'
                 encoding = 'cp949', skiprows = 15, thousands=',')
#OS가 Windows일 경우 engine='python'을 입력해야 합니다.
#국토부가 제공하는 실거래가 자료는 'cp949'로 encoding되어있습니다. 
#실거래가 자료는 첫 15줄에 경고문이 쓰여있어, skiprows를 통해 16번째 줄부터 불러오도록 해야 합니다.
#실거래가 자료의 거래금액(만원) 항목에는 천 단위로 ',' 표시가 있어서 thousands를 통해 정제해야 합니다.

In [ ]:
아파트

In [ ]:
분석시기 = ['201708', '201808', '201908']
아파트 = pd.DataFrame()
#pd.DataFrame()은 빈(empty) DataFrame을 만드는 명령어입니다.
for 시기 in 분석시기:
    임시저장 = pd.read_csv('아파트(매매)__실거래가_{}.csv'.format(시기), engine='python',
                 encoding = 'cp949', skiprows = 15, thousands=',')
    #'따옴표' 뒤에 붙는 .format(변수)를 넣으면 <for ~ in> 안에서 같은 형식 안에 여러 변수를 입력할 수 있습니다.
    아파트 = pd.concat([아파트, 임시저장], sort=False)
    #pd.concat은 여러 DataFrame을 합하는 명령어입니다.

In [ ]:
아파트

In [ ]:
#서울 강남구에 있는 아파트 추리기
서울강남구 = 아파트.loc[아파트['시군구'].str.contains('서울특별시 강남구')].reset_index(drop=True)
#.str.contains('단어')는 '단어'가 포함된 열을 반환하도록 하는 명령어
#.reset_index(drop=True)는 반환된 행 번호를 재배치하는 명령어

In [ ]:
서울강남구

In [ ]:
#.groupby()는 피봇테이블과 유사한 명령어
서울강남구.groupby('계약년월').mean()
#groupby() 뒤에 .mean()을 붙이면 각 열(항목)별 평균값을 반환함

In [ ]:
#서울 강남구 아파트 중 특정 단지(=은마) 아파트와 면적을 검색함
은마76 = 서울강남구.loc[(서울강남구['단지명'] == '은마')
              & (서울강남구['전용면적(㎡)'] == 76.79)]
#'&' 명령어는 and 조건, '|' 명령어는 or 조건.
#'==' 명령어는 일치, '!=' 명령어는 불일치 조건.

In [ ]:
#groupby() 뒤에 .count()를 붙여서 개수(=거래량)를 확인함.
은마76.groupby('계약년월').count()

In [ ]:
#pd.pivot_table()에서 평균값이 아닌 합계, 개수 등을 구하기 위해서 필요한 모듈
# !pip install numpy
import numpy as np

In [ ]:
서울 = 아파트.loc[아파트['시군구'].str.startswith('서울')].reset_index(drop=True)
서울['자치구'] = 서울['시군구'].str.split().str[1]
#.str.startswith('단어')는 '단어'로 시작하는 열을 반환하도록 하는 명령어
#.str.split()은 띄어쓰기로 단어를 구분하여 묶음으로 저장하는 명령어
#.str[1]은 구분된 묶음 중 두 번째 단어를 선택하도록 하는 명령어

In [ ]:
서울

In [ ]:
#초간단 피봇테이블 생성하기
결과 = pd.pivot_table(서울, values='거래금액(만원)', index='자치구', columns='계약년월')
결과

In [ ]:
#.sort_values(by='기준')은 기준값으로 재정렬하는 명령어
결과.sort_values(by=201908, ascending=False)
#ascending=False는 오름차순의 반대, 즉 내림차순으로 정렬하는 명령어

In [ ]:
결과 = pd.pivot_table(서울, values='거래금액(만원)', index='자치구', columns='계약년월')
결과.to_csv('아파트_실거래가_서울자치구별_201708-201908.csv')
#.to_csv('파일명')로 결과값을 파일로 저장
#결과.to_excel('아파트_실거래가_서울자치구별_201708-201908.xlsx')

In [ ]:
#단지별 고유값을 만들기 위한 'indexing'(행의 고유화) 작업
서울['기준'] = 서울['시군구'] + ' ' + 서울['번지'] + ' ' + 서울['단지명']
#문자열은 '+'를 이용해 서로 합칠 수 있음. ' '는 띄어쓰기(공백)을 의미함.

In [ ]:
#'기준'으로 피봇테이블 생성
결과_단지별 = pd.pivot_table(서울, values='거래금액(만원)', index='기준', columns='계약년월')
결과_단지별['증가율(최근1년)'] = 결과_단지별[201908] / 결과_단지별[201808] - 1
결과_단지별['증가율(최근2년)'] = 결과_단지별[201908] / 결과_단지별[201708] - 1
#1년전, 2년전 대비 2019년 8월의 증가율 산출하기

In [ ]:
#.isnull()은 값이 존재하지 않는 열을 반환하도록 하는 명령어
#그 뒤에 != True를 붙였기 때문에 값이 존재하는 열을 반환하도록 함.
결과_단지별 = 결과_단지별.loc[(결과_단지별['증가율(최근2년)'].isnull() != True)
                   & (결과_단지별['증가율(최근1년)'].isnull() != True)]

In [ ]:
#최근 2년간 증가율을 기준으로 내림차순 정렬하기
결과_단지별.sort_values(by='증가율(최근2년)', ascending=False)

In [ ]:
#대우트럼프월드를 서울 DataFrame에서 다시 검색해보기
서울.loc[서울['단지명'] == '대우트럼프월드']

In [ ]:
#예제 풀어보기
#1. 서울 마포구에 있는 아파트 추려내기

In [ ]:
#2. 최근 3년간 마포구에서 가장 비쌌던 아파트는?

In [ ]:
#3. 전용면적 40 이하 마포구 아파트의 평균 가격은?

In [ ]:
#4. 시군구, 번지, 단지명, 전용면적으로 <‘기준’ 열> 만들기

In [ ]:
#5. 마포구에서 최근 2년간 증가율이 가장 높았던 주택은?

# Pandas : 가계금융복지조사 마이크로데이터 분석

In [ ]:
import pandas as pd
import numpy as np
# !pip install math
import math

In [ ]:
가금복 = pd.read_csv('가계금융복지조사2018.csv', header=None)
포맷 = pd.read_excel('가계금융복지조사2018_포맷.xls')
가금복.columns = 포맷['항목명'].str.strip().tolist()

In [ ]:
포맷

In [ ]:
가금복

In [ ]:
# 30세 미만, 미혼, 전세or월세 거주
청년가구 = 가금복.loc[(가금복['가구주 만나이'] < 30) 
               & (가금복['가구주 혼인상태'] == 1)
               & (가금복['입주형태 통합별'] != 'G1')
               & (가금복['입주형태 통합별'] != 'G4')].reset_index(drop=True)
print(청년가구['가중치'].sum())

In [ ]:
# 만 65세 초과, 노인 1인가구의 평균 가처분소득
노인가구 = 가금복.loc[(가금복['가구주 만나이'] > 65)
               & (가금복['가구원수'] == 1)].reset_index(drop=True)
노인가구['임시:가중치_계산'] = 노인가구['처분가능소득'] * 노인가구['가중치']
print(노인가구['임시:가중치_계산'].sum() / 노인가구['가중치'].sum())

In [ ]:
# 여성 1인가구 중 기대 최소생활비보다 가처분소득이 적은 가구의 비율
여성가구 = 가금복.loc[(가금복['가구주 성별'] == 2)
               & (가금복['가구원수'] == 1)
               & (가금복['가구주 미은퇴시_최소생활비'] > 0)].reset_index(drop=True)
여성가구['임시:기대이하소득'] = (여성가구['처분가능소득'] < 여성가구['가구주 미은퇴시_최소생활비'] * 12)
# 부등호(<) 조건으로 True 혹은 False 값을 임시 열에 저장
print(여성가구.loc[여성가구['임시:기대이하소득'] == True]['가중치'].sum() 
      / 여성가구['가중치'].sum())

In [ ]:
# 제곱근 방식(한국 통계청 방식)의 가구균등화지수 생성하기
가구균등화지수 = {}
# 1부터 10까지 for ~ in 명령어 실행
for 숫자 in range(1, 11): 
    가구균등화지수['{}인가구'.format(str(숫자))] = math.sqrt(숫자)
    #str(변수)는 변수를 문자로 변환하는 명령어
    #math.sqrt(숫자)는 제곱근을 반환하는 명령어
print(가구균등화지수)

In [ ]:
가금복['균등화소득'] = 0
for 숫자 in range(1, 11):
    가금복['균등화소득'] = np.where(가금복['가구원수'] == 숫자,
                            가금복['가구소득(경상소득)'] / 가구균등화지수['{}인가구'.format(str(숫자))], 
                            가금복['균등화소득'])

In [ ]:
가금복['균등화소득'] = 0
# 1인가구를 기준으로 경상소득을 균등화하기
for 숫자 in range(1, 11):
    가금복['균등화소득'] = np.where(가금복['가구원수'] == 숫자,
                            가금복['가구소득(경상소득)'] / 가구균등화지수['{}인가구'.format(str(숫자))], 
                            가금복['균등화소득'])
    #np.where(조건, True일 경우 부과할 값, False일 경우 부과할 값) 명령어의 구조
    #1인부터 10인까지 for ~ in 루프를 통해 가구원수를 확인하고 균등화지수를 적용하는 과정
    #해당 가구원수가 아닐 경우 이미 입력된 균등화소득을 그대로 둠.

In [ ]:
가금복 = 가금복.sort_values(by='균등화소득')
가금복['가중치_누적합계'] = 가금복['가중치'].cumsum()
기준 = 가금복['가중치'].sum() / 2
중위소득 = 가금복.loc[가금복['가중치_누적합계'] >= 기준].iloc[0]['균등화소득']

In [ ]:
# 균등화소득을 기준으로 오름차순 정렬
가금복 = 가금복.sort_values(by='균등화소득')
가금복['가중치_누적합계'] = 가금복['가중치'].cumsum()
# .cumsum()은 해당 행 위치에서의 누적합계를 구하는 명령어
기준 = 가금복['가중치'].sum() / 2
# 중위값은 가중치 합계의 절반을 의미함
중위소득 = 가금복.loc[가금복['가중치_누적합계'] >= 기준].iloc[0]['균등화소득']
# 가중치의 누적합계가 기준과 일치하거나 큰 묶음을 추라는 과정
# 묶음을 추린 후, .iloc[0] 명령어를 통해 가장 첫 번째 행을 선택함.
# 해당 목록에서 ['균등화소득'] 값을 불러와서 중위소득 변수에 저장함.
print(중위소득 / 12)
# 경상소득은 연단위(만원) 소득이기 때문에 월 단위로 환산함.

In [ ]:
#예제 풀어보기
#1. 가구주가 만 50세 이상이며, 입주유형이 자가소유(='G1')인 가구의 합계 구하기

In [ ]:
#2. 만 39세 이하 1인 청년가구의 평균 사적이전소득 구하기

In [ ]:
#3. 남성이 가구주인 가구 중 기대 적정생활비보다 처분가능소득이 높은 비율 구하기

In [ ]:
#4. (균등화) 처분가능소득 기준 상위 1%의 경계값 구하기